## META TGI

In [ ]:
# pip install -U sagemaker transformers sentencepiece
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

In [111]:
import os
import time
from sagemaker.huggingface import get_huggingface_llm_image_uri
import boto3
import json
from sagemaker.huggingface import HuggingFaceModel
import argparse
import yaml
import sagemaker

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('meta-llama/Llama-2-7b-chat-hf')

In [113]:
llm_image = get_huggingface_llm_image_uri("huggingface", version="1.1.0")
    # TGI config
config = {
    "HF_MODEL_ID": "TheBloke/Llama-2-70B-GPTQ",  # model_id from hf.co/models
    "SM_NUM_GPUS": json.dumps(4),  # Number of GPU used per replica
    "MAX_INPUT_LENGTH": json.dumps(3072),  # Max length of input text
    "MAX_TOTAL_TOKENS": json.dumps(4096),  # Max length of the generation (including input text)
    "HUGGING_FACE_HUB_TOKEN": "HF KEY",
    "HF_MODEL_QUANTIZE" : "gptq"
}

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [114]:
llm_model = HuggingFaceModel(role=sagemaker.get_execution_role(), image_uri=llm_image, env=config)
endpoint_region = llm_model.sagemaker_session._region_name
credentials = llm_model.sagemaker_session.boto_session.get_credentials()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [115]:
endpoint_name=f"Llama2-70b-gptq-{str(time.time()).split('.')[-1]}"
llm = llm_model.deploy(
            initial_instance_count=1,
            instance_type="ml.g5.12xlarge",
            container_startup_health_check_timeout=300,
            endpoint_name=endpoint_name
        )

------------!

In [116]:
prompt="""Generate a summary of the following:
As I sit down to write my second annual shareholder letter as CEO, I find myself optimistic and energized by what lies ahead for Amazon. Despite 2022 being one of the harder macroeconomic years in recent memory, and with some of our own operating challenges to boot, we still found a way to grow demand (on top of the unprecedented growth we experienced in the first half of the pandemic). We innovated in our largest businesses to meaningfully improve customer experience short and long term. And, we made important adjustments in our investment decisions and the way in which we’ll invent moving forward, while still preserving the long-term investments that we believe can change the future of Amazon for customers, shareholders, and employees.

While there were an unusual number of simultaneous challenges this past year, the reality is that if you operate in large, dynamic, global market segments with many capable and well-funded competitors (the conditions in which Amazon operates all of its businesses), conditions rarely stay stagnant for long.

In the 25 years I’ve been at Amazon, there has been constant change, much of which we’ve initiated ourselves. When I joined Amazon in 1997, we had booked $15M in revenue in 1996, were a books-only retailer, did not have a third-party marketplace, and only shipped to addresses in the US. Today, Amazon sells nearly every physical and digital retail item you can imagine, with a vibrant third-party seller ecosystem that accounts for 60% of our unit sales, and reaches customers in virtually every country around the world. Similarly, building a business around a set of technology infrastructure services in the cloud was not obvious in 2003 when we started pursuing AWS, and still wasn’t when we launched our first services in 2006. Having virtually every book at your fingertips in 60 seconds, and then being able to store and retrieve them on a lightweight digital reader was not “a thing” yet when we launched Kindle in 2007, nor was a voice-driven personal assistant like Alexa (launched in 2014) that you could use to access entertainment, control your smart home, shop, and retrieve all sorts of information.

There have also been times when macroeconomic conditions or operating inefficiencies have presented us with new challenges. For instance, in the 2001 dot-com crash, we had to secure letters of credit to buy inventory for the holidays, streamline costs to deliver better profitability for the business, yet still prioritized the long-term customer experience and business we were trying to build (if you remember, we actually lowered prices in most of our categories during that tenuous 2001 period). You saw this sort of balancing again in 2008-2009 as we endured the recession provoked by the mortgage-backed securities financial crisis. We took several actions to manage the cost structure and efficiency of our Stores business, but we also balanced this streamlining with investment in customer experiences that we believed could be substantial future businesses with strong returns for shareholders. In 2008, AWS was still a fairly small, fledgling business. We knew we were on to something, but it still required substantial capital investment. There were voices inside and outside of the company questioning why Amazon (known mostly as an online retailer then) would be investing so much in cloud computing. But, we knew we were inventing something special that could create a lot of value for customers and Amazon in the future. We had a head start on potential competitors; and if anything, we wanted to accelerate our pace of innovation. We made the long-term decision to continue investing in AWS. Fifteen years later, AWS is now an $85B annual revenue run rate business, with strong profitability, that has transformed how customers from start-ups to multinational companies to public sector organizations manage their technology infrastructure. Amazon would be a different company if we’d slowed investment in AWS during that 2008-2009 period.

Change is always around the corner. Sometimes, you proactively invite it in, and sometimes it just comes a-knocking. But, when you see it’s coming, you have to embrace it. And, the companies that do this well over a long period of time usually succeed. I’m optimistic about our future prospects because I like the way our team is responding to the changes we see in front of us.

Over the last several months, we took a deep look across the company, business by business, invention by invention, and asked ourselves whether we had conviction about each initiative’s long-term potential to drive enough revenue, operating income, free cash flow, and return on invested capital. In some cases, it led to us shuttering certain businesses. For instance, we stopped pursuing physical store concepts like our Bookstores and 4 Star stores, closed our Amazon Fabric and Amazon Care efforts, and moved on from some newer devices where we didn’t see a path to meaningful returns. In other cases, we looked at some programs that weren’t producing the returns we’d hoped (e.g. free shipping for all online grocery orders over $35) and amended them. We also reprioritized where to spend our resources, which ultimately led to the hard decision to eliminate 27,000 corporate roles. There are a number of other changes that we’ve made over the last several months to streamline our overall costs, and like most leadership teams, we’ll continue to evaluate what we’re seeing in our business and proceed adaptively.

We also looked hard at how we were working together as a team and asked our corporate employees to come back to the office at least three days a week, beginning in May. During the pandemic, our employees rallied to get work done from home and did everything possible to keep up with the unexpected circumstances that presented themselves. It was impressive and I’m proud of the way our collective team came together to overcome unprecedented challenges for our customers, communities, and business. But, we don’t think it’s the best long-term approach. We’ve become convinced that collaborating and inventing is easier and more effective when we’re working together and learning from one another in person. The energy and riffing on one another’s ideas happen more freely, and many of the best Amazon inventions have had their breakthrough moments from people staying behind after a meeting and working through ideas on a whiteboard, or continuing the conversation on the walk back from a meeting, or just popping by a teammate’s office later that day with another thought. Invention is often messy. It wanders and meanders and marinates. Serendipitous interactions help it, and there are more of those in-person than virtually. It’s also significantly easier to learn, model, practice, and strengthen our culture when we’re in the office together most of the time and surrounded by our colleagues. Innovation and our unique culture have been incredibly important in our first 29 years as a company, and I expect it will be comparably so in the next 29.

A critical challenge we’ve continued to tackle is the rising cost to serve in our Stores fulfillment network (i.e. the cost to get a product from Amazon to a customer)—and we’ve made several changes that we believe will meaningfully improve our fulfillment costs and speed of delivery."""

In [117]:
# # Number of input tokens
# tok=tokenizer.encode(prompt, return_tensors="pt")[0]
# len(tok)

1720

In [118]:
## Initialize sagemaker endpoint
from sagemaker.predictor import Predictor
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer
predictor=Predictor(endpoint_name=endpoint_name,
                    serializer=JSONSerializer(),                    
    deserializer=JSONDeserializer(),
                   )

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [119]:
%%time
predictor.predict({"inputs": prompt,
                      "parameters": {
            "max_new_tokens": 200,
            # "top_p": 0.9,
            # "temperature": 0.6,
            "return_full_text": False,
        }})

CPU times: user 1.26 ms, sys: 11.6 ms, total: 12.8 ms
Wall time: 27.4 s


[{'generated_text': ' We’ve been building out our fulfillment network for over 25 years, and we’ve grown from a single fulfillment center in Seattle in 1997 to over 1,000 fulfillment centers around the world today. We’ve also grown from a single fulfillment center in Seattle in 1997 to over 1,000 fulfillment centers around the world today. We’ve also grown from a single fulfillment center in Seattle in 1997 to over 1,000 fulfillment centers around the world today. We’ve also grown from a single fulfillment center in Seattle in 1997 to over 1,000 fulfillment centers around the world today. We’ve also grown from a single fulfillment center in Seattle in 1997 to over 1,000 fulfill'}]

In [ ]:
import boto3

def delete_endpoint(endpoint_name):
    sm_client = boto3.client('sagemaker')
    try:
        # Get the endpoint configuration name first
        response = sm_client.describe_endpoint(EndpointName=endpoint_name)
        endpoint_config_name = response['EndpointConfigName']

        # Delete the endpoint
        # sm_client.delete_endpoint(EndpointName=endpoint_name)
        print(f"Endpoint {endpoint_name} deleted successfully.")

        # Delete the endpoint configuration
        sm_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
        print(f"Endpoint configuration {endpoint_config_name} deleted successfully.")
    except Exception as e:
        print(f"Error deleting endpoint {endpoint_name} and its configuration: {str(e)}")

# Assuming 'config_file' is a list of dictionaries with 'endpoint' and 'instance' keys
delete_endpoint(endpoint_name)
